In [1]:
import datetime
import pandas as pd

purchased_item =[]  # A list about purchased drinks for only one transaction.It becomes empty when machines is restarted.
coin=[]    #  A list that stores sales data for eachl buys  of one transaction. It becomes empty when machines is restarted.
moneyrecord ={}   # keys are date-time, values are transaction amount at that time. Keeps record of all transactions.

## Functions Related to User and User/ Customer Requirements

In [2]:
def restart():
    '''
    Asking user to restart for a new transaction or logging in the machine or cancel out. 
    Confirms resetting previous transaction history if restarted.
    Generating welcome message after restarting, else send goodbye message. 
    '''
    print('Machine is in good condition')
    
    while True:
        print(f"Latest purchased items were {purchased_item} and total cost was {totalcost()}")
        answer=input("Please press 'y' for restarting and commence a new transaction" +"\n"+
                     "For logging in the machine press'l'"+"\n"+
                      "For cancellling transaction press 'n'                                       ").lower()
        if answer not in ('y', 'n','l'):
            print("Invalid input. select 'y' or 'l' or 'n'")
            restart()
        if answer == 'y':
            purchased_item.clear()
            coin.clear()
            print(f'Current purchased items are {purchased_item} and total cost is {totalcost()}')
            print('Welcome to the vending machine.'+'\n'+ 
                  'This machine accepts coins from 10 cents to 2 dollars only.')
            show_menu()
        elif answer=='l':
            log_in()
        else:
            print("Goodbye")
            restart()

In [3]:
 '''
Initial Setting of the machine is performed from the initial_inventory data provided by the machine owner (customer) 
who will decide the stock quantity and pricing of the drinks.
'''

def set_machine(stockormenu):
    '''
setmachine function takes one argument that can have values:'stock' or 'menu'. 
Input of the data is taken from machine owner by a text file named initial_inventory.
    '''
    file = open("initial_inventory.txt", "r") # opening a text file that contains information about 
                                              # drinks,price and initial stock
    keys =[]
    price =[]
    stock =[]
    
# filling up the empty key, price and stock list from initial_inventory text file
    for aline in file:
        values = aline.split()
        keys.append(values[0])
        price.append(float(values[1]))
        stock.append(float(values[2]))
    file.close()
    if stockormenu=='stock':
        return dict(zip(keys,stock))     # creating dictionary with drinks:stock combination
    elif stockormenu=='menu':
        return dict(zip(keys,price))# creating dictionary wiith drinks:price combination
    else:
        print('not the correct paramenter. select either stock or menu')
        set_machine(stockormenu)

In [4]:
mainmenu = set_machine('menu')       # creating a menu dictionary showing the price of all drinks 
main_stock = set_machine('stock')    # creating a dictionary that shows stock quantity of different drinks

In [5]:
def displaymenu():
    '''
    the function creates a customized menu that provides information when an item has zero stock.
    it also tells how the price of sugar and milk will be implemented.
    
    '''
    for item in mainmenu:
        if main_stock[item] ==0:
            print(f'Sorry, no {item} is available today !')
        else:
            if item == 'sugar' or item == 'milk':
                print(f'{item} price is $ {mainmenu[item]} extra if you choose for tea or coffee')
            else:
                print(f'{item} price is  $ {mainmenu[item]}')
    

In [6]:
def show_menu():
    '''
    Offering the user to get the menu on the screen with the option to leave the transaction.
    '''
    
    menuwant=input('Do you want to see menu now? then press "y"'+'\n'+
                   'Or want to cancel? press any key                                                ').lower()
    if menuwant == 'y':
            print('Here is the menu for your drinks ')
            displaymenu()
            drink_selection()
    else:
        print("Goodbye")
        change_func()
    

In [7]:
def totalcost():
    '''
    determines the total cost of the purchased drinks in single transaction.
    '''
    cost =0
    for i in purchased_item:
        cost = cost+mainmenu[i]
    return cost

In [8]:
def deposit():
    '''
    calculate the coins inserted in the machine in each time in a transaction.
    It works on a list 'coin' , which is developed everytime user inserts a coin in 
    the machine. List becomes empty when machine is restarted.
    '''
    coins=0
    for i in range(len(coin)):
        coins=coins+coin[i]
    return coins

In [9]:
def low_balance(items):
    '''
     Provides option when user in low balance while purchasing a drink. Options are: continue buyeing,
    or changing drinks or terminating current transactions. With in the function four more functions are recalled.
    
    '''
    transact= input('You are in low balance. Do you wanna continue buy? Then press "y".'+'\n'+
                    'To change your order press "ch" '+'\n'+
                    'To cancel transaction here or for no response, press any key except y & ch         ').lower()
    if transact == 'y':
        
        try:
            morecoins = float(input(f'You need {totalcost()-deposit()+mainmenu[items]} dollars more to buy {items}.'
                                +'\n'+ 'Please enter more coins:                                 '))/100
            coin_validity(morecoins)  # crosschecking the validity of the coins.
            if items =='milk':                                
                milk()
            elif items =='sugar':
                sugar()
            else:
                drinks_func(items)      
            
        except:
            print('Please enter integer numbers only. Confirm your buyeing again.')
            low_balance(items)
            
    elif transact=='ch':
        another_drink() # provides option to select different drinks
    else:
        change_func()  # provides option to terminate the transaction  

In [10]:
def milk():
    '''
    User can select milk or not. Cross checking of costing with available balance is confirmed.
    Purchasing and stock list is updated.
    '''
    print('wait please. Water is boiling'+'\n'+'Would you like to add milk?')
    if str(input('Do you need milk? Press "y" or "n"                                                  ').lower())=='y':
        if deposit()-totalcost()>= mainmenu['milk']:
            main_stock['milk'] = main_stock['milk']-1   # updating stock list
            purchased_item.append('milk')               # updating purchase list
            print('ok, wait for a second!It will not take long. We have just added milk for you.')
            sugar()
        else:
            print('You do not have enough deposit. Reconfirm adding milk please.')
            low_balance('milk')            
    else:
        print('ok, no milk.')
        sugar()
    

In [11]:
def sugar():
    '''
    User can select sugar or not. Cross checking of costing with available balance is confirmed.
    Purchasing and stock list is updated. Customer can select sugar either manually or auto option.
    '''
    
    sugar_add =str(input('Do you need to add sugar automatically?'+'\n'+
                        'Press "y" for yes, any key for no or "manual", if you do by yourself             ').lower())

    if sugar_add =='y' or sugar_add =='manual':
        if deposit()-totalcost()>= mainmenu['sugar']:
            main_stock['sugar'] = main_stock['sugar']-1 # updating stock list
            purchased_item.append('sugar')  # updating purchased list
            if sugar_add=='y':
                print('ok, wait for a second!It will not take long. we are adding sugar automatically.')
            else:
                print('Your drink is stored. Collect sugar. Then add manually')
        else:
            print('You do not have enough deposit. Reconfirm adding sugar please.')
            low_balance('sugar')       
    else:    
        print('ok, no sugar.')

In [12]:
def change_func():
    '''
    while ending the transaction, this function updates the all transaction record and then provide 
    statistical data: bought items, summation of all costs and change amount.
    '''
    change = deposit()-totalcost()
    if purchased_item ==[]:
        print(f"You paid {deposit()} dollars and you bought nothing" +'\n'+
              f'Thank you. Hope to buy next time. Your change is {change} dollars ')
        print("Take your change please")
        print("Thank you.")
    else:
        moneyrecord[datetime.datetime.now().strftime("%d/%m/%Y_%H:%M:%S")] = totalcost() # updating sales data
        
        print(f"Wait for a second. Collect your drinks. You paid ${deposit()} and you bought this drinks:{purchased_item}" 
              +'\n'+ f"Your total purchased amount is {totalcost()} dollars. So Your change is: {change} dollars")
        print("Take your change please")
        print("Thank you for buyeing. Goodbye")
        restart()

In [13]:
def another_drink():
    j= str(input('Do you want another drink: y or n?                                              ')).lower()
    '''
    allowing user to choose another drink at any stage of the transaction
    '''
    if j=='y':
        displaymenu()
        print(drink_selection()) # selecting new drinks 
    else:
        print(change_func()) # going for ending transaction.
    

In [14]:
def no_stock():
    '''
    if user still selects an item that is not available after mentioning in menu
    '''
    print('We are sorry, the item is not in stock; do you need another drink?')
    item= str(input('please select "y" for yes'+'\n'+
                    'Or press any key to terminate :                                            ')).lower()
    if item =='y':
        drink_selection() # selecting new drinks
    else:
        change_func()  # going for ending transaction.
            

In [15]:

def not_in_menu():
    '''
    If a customer choose an item that is not in the menu. Provide options to choose any listed drinks or to leave.
    '''
    print('Sorry, your selection is not in the list.'+'\n'+
                  'Please select only listed drinks from the menu.')
    reselect= str(input('Do you want another drink? please select "y" for yes'+'\n'+
                        'Or press any key to terminate :                                              ')).lower()
    if reselect =='y':
        drink_selection() # selecting new drinks
    else:
        change_func()


In [16]:
def coin_validity(penny):
    '''
    cross-checking the inserted coins with the allowed coin list
    '''
    
    coinlist = [.1,.2,.5,1,2] # allowed coin list
    if penny not in coinlist:
        print('Sorry, we can take only specific coins.')
        try:
            penny = (float(input('Please enter your coins. Allowable coins are:'+'\n'+
                            '10cents,20cents,50cents,100cents,200cents                             '))/100)
        except:
            print('Please insert only coins. So integer number is expected. Try again.')
            coin_validity(penny)
    coin.append(penny)


In [17]:
def drink_selection():
    '''
    Providing option to the user for selecting drinks mentioning the cost, or cancelling transaction.
    '''
    item = str(input("Please select your drinks. For cancellation, press 'cancel':                       ")).lower()
    if item!='cancel':
        if item in mainmenu:
            if main_stock[item]!=0:
                print(f'Cost of your selected drink: {item} is ${mainmenu[item]}') #Cost of the selected item is displayed
                drink_confirmation(item)
            else:
                no_stock()   
        else:
            not_in_menu()       
    else:
        change_func()
    

In [18]:
def drink_confirmation(item):
    '''
    Provide option for continuing through crosschecking of coin validation or order changing or cancellation.
    Function also doing error handling.
    '''                                    
    confirm = str(input('Do you wanna proceed with your buys? press "y"'+'\n'+ 
                        'Otherwise press any key                                                     ')).lower()
        
    if confirm == 'y':
        try:
            initialcoins = (float(input('Please enter your coins. Allowable coins are:'+'\n'+
                                    '10cents,20cents,50cents,100cents,200cents                         '))/100)
        except:
            print('Please enter integer numbers only. Try again')
            try:
                initialcoins = (float(input('Please enter your coins. Allowable coins are:'+'\n'+
                                    '10cents,20cents,50cents,100cents,200cents                         '))/100)
            except:
                print('This is second time. Please enter integer numbers only. Now confirm your buyeing again')
                drink_confirmation(item)
                
        coin_validity(initialcoins)
        
        drinks_func(item)
        
    else: 
        print('Do you want to change order? Then select new item. You just got menu. ')
        drink_selection() 
       

In [19]:
def drinks_func(item):
    '''
    Updating stock quantity. Choosing milk and sugar. choosing another drinks and dealing with 
    low balance by recalling corresponding functions.
    '''
    if deposit()-totalcost()>= mainmenu[item]:
        furtherConfirm = input('Just dispensing. Do you like to continue? press any key'+'\n'+
                                'or take your money back? press "b".........                            ').lower()
        if furtherConfirm !='b':       
            main_stock[item] = main_stock[item]-1
            purchased_item.append(item)
        
                                                
            if item in ['tea','coffee']:       # choosing milk and sugar for tea and coffee.
                milk()
                
            print(f'So far total costing is {totalcost()}'+'\n'+ 
                  f'So far, you purchased {purchased_item} drinks.'+'\n'
                  f'and your current balance is {deposit()-totalcost()} dollars')  
                
            another_drink()     # Choosing another drink
        else:
            change_func()
    else:
        low_balance(item)   # if user is on low-balance 
                              
          


# Functions Related to Customer (Owner) Requirements

In [20]:
def log_in():
    '''
    Function provides opportunity to login as an owner with password 
    or can continue as an user. Password is : 123
    '''
    logcatagory= input('Do you want to login as an owner? Press "red" button to insert password'+'\n'+
                       'Otherwise, press any key.........................................       ').lower()
    if logcatagory=='red':
        passWord= input('Insert your customer login password...........                   ')
        if passWord=='123':
            customer()
        else:
            print('Wrong password. Try again')
            log_in()
    else:
        restart()

In [21]:
def stock_alert():
    '''
    Generating an alert due to stock shortage of any drinks
    '''
    for stock in main_stock: 
        if main_stock[stock]==0:
            print(f'ALERT! THER IS NO {stock} IN THE MACHINE. PLEASE REFILL NOW.')
        

In [22]:

def factory_reset():
    '''
    For factory reset, all kind of data(transaction, stock) will be reset to zero.
    '''
    
    print('Attention, Machine is resetting to factory re-set.' +'\n'+
          'Please take the record of your business transactions. All data shall be erased')
    purchased_item.clear()
    coin.clear()
    main_stock.clear()
    moneyrecord.clear()
    print('The machine has been reset'+'\n'+ 'Good to run now')
   

In [23]:
def reset():
    '''
    Allows to reset the or stop the machine with password or can continue running.
    '''
    print('Continue running? Press any key in the following password option.'+ '\n'+
          'To reset the machine, please insert your password again.')
    password =input('Insert your password here.....................                      ') # confirming owners login
    if password=='123':
        response = input('Do you want a factory re-set? press f' +'\n'+
                         'Do you need to stop the machine for mintenance, press x'+'\n'+
                         'Or continue running? press any key                                          ').lower()
        if response == 'f':
            factory_reset()
            print('The machine has been reset'+'\n'+ 'Good to run now')
        elif response=='x':
            print('Sorry, the machine is not in service'+'\n'+ 'Goodbye')
            reset()
        else:
            print('Machine is in good condition. It is running smoothlyly')
    else:
        print('Machine is in good condition. It is running smoothlyly')
        


In [24]:

def owner_record():
    '''
    Creating csv files that will record all sales data and updated stock.
    '''
    recordFile = pd.DataFrame(moneyrecord.items(), columns=['Date_Time', 'Sales_history ($)'])
    latest_stock_details = pd.DataFrame().append(main_stock, ignore_index=True)
    print(latest_stock_details)
    print(recordFile)
    recordFile.to_csv("sales_records", sep='\t', encoding='utf-8')
    latest_stock_details.to_csv("latest_inventory", sep='\t', encoding='utf-8')

In [25]:
def customer():
    '''
    This is the ultimate function which recalls other functions related to customer requirement.
    Provides stock alert data, all transaction records on request and resetting the machine if requested.
    '''
    print('Welcome to the vending machine.'+'\n'+ 
          'Machine is in good condition. It is running smoothlyly'+ '\n'+ 
          'This machine accepts coins from 10 cents to 2 dollars only.')
    
    stock_alert()
    search_record = input('Do you need the updated sales and stock data? y or n                    ').lower()
    
    if search_record=='y':
        print('You can see latest inventory and sales history here.''\n'+
              ' You will also have "latest_inventory" and "sales_records" files as backup. Please check')
        owner_record()
        
    else:
        print('Ok. you can have that later.') 
    
    reset()
    
    logOut = input('do you want to logout and check the machine as user? y or n                     ').lower()
    if logOut=='y':
        restart()
    else:
        customer()
    

## Testing the whole program

In [ ]:
# For testing the whole program, please run the whole notebook.
#login Password is : 123
# Please replace the word 'press' by 'write' to do any user input.
# For 'no', maximum time it is any key.
log_in()

Do you want to login as an owner? Press "red" button to insert password
Otherwise, press any key.........................................       y
Machine is in good condition
Latest purchased items were [] and total cost was 0
Please press 'y' for restarting and commence a new transaction
For logging in the machine press'l'
For cancellling transaction press 'n'                                       y
Current purchased items are [] and total cost is 0
Welcome to the vending machine.
This machine accepts coins from 10 cents to 2 dollars only.
Do you want to see menu now? then press "y"
Or want to cancel? press any key                                                y
Here is the menu for your drinks 
coke price is  $ 2.0
juice price is  $ 2.0
tea price is  $ 2.5
coffee price is  $ 2.5
Sorry, no fanta is available today !
milk price is $ 0.5 extra if you choose for tea or coffee
sugar price is $ 0.5 extra if you choose for tea or coffee
Please select your drinks. For cancellation, press 'c